In [2]:
import math
import cmath 
import numpy as np
import matplotlib.pyplot as plt
from skrf import Network, Frequency


from IPython.display import Image  
%matplotlib inline
params = {'legend.fontsize': 24,
          'figure.figsize': (10, 10),
         'axes.labelsize': 24,
         'axes.titlesize': 24,
         'xtick.labelsize':24,
         'ytick.labelsize':24}
plt.rcParams.update(params)




In [56]:
def Linvill(yt):
    y11 = yt[0,0]
    y12 = yt[0,1]
    y21 = yt[1,0]
    y22 = yt[1,1]
    C = (abs(y12*y21))/(2*y11.real*y22.real-(y12*y21).real)
    print('###############  LINVILL   #####################')
    print( 'coeficiente de linvill C= {:4.2F}' .format(C))
    if C < 1:
        print( 'Es incondicionalmente estable')
    else :
        print( 'Es potencialmente inestable')
    return C

def Stern(yg, yl, yt):
    y11 = yt[0,0]
    y12 = yt[0,1]
    y21 = yt[1,0]
    y22 = yt[1,1]
    K = (2*(y11.real+yg.real)*(y22.real+yl.real)) / (abs(y12*y21)+(y12*y21).real)
    print('###############  STERN   #####################')
    print( 'coeficiente de stern K= {:4.2F}' .format(K))
    if K > 1:
        print( 'Estable')
    else :
        print( 'Inestable')
      
    yin  = y11 - (y21*y12)/(y22+yl)
    yout = y22 - (y21*y12)/(y11+yg)
    print( 'yin  = {0.real:4.3F} {0.imag:4.2E}' .format(yin) )
    print( 'rin  = {0.real:4.3F} ' .format(1/yin.real) )
    
    print( 'yout = {0.real:4.3F} {0.imag:4.2E}' .format(yout) )
    print( 'rout = {0.real:4.3F} ' .format(1/yout.real) )
    
    
    Gg = yin.real/(yg+yin).real
    G  = abs(y21)**2 *yl.real /(yin).real  /(y22+yl).real**2
    Gt = 4* abs(y21)**2 *yl.real *yg.real /abs((y11+yg)*(y22+yl)-y21*y12)**2
    
    print( 'Gg= {0.real:4.2F} ' .format((Gg)))
    print( 'G = {0.real:4.2F} ' .format((G )))
    print( 'Gt= {0.real:4.2F} dB' .format((10*np.log10(Gt) )))





def neutraliza(yt,wo,n = 1):
    Cn = -yt[0,1].imag/wo*n
    y11 =  wo*Cn*1j
    y12 =  wo*Cn*1j/n
    y21 =  wo*Cn*1j/n
    y22 =  wo*Cn*1j/n**2
    yn  = np.matrix([[y11, y12],[ y21, y22]])
    # cuadripolo resultante
    ytn  = yt + yn
    print('cap=',Cn, 'n=', n)
    return (ytn,Cn)


def realimenta(yt,wo,yf):
    y11 =   yf
    y12 =  -yf
    y21 =  -yf
    y22 =   yf  
    yn  = np.matrix([[y11, y12],[ y21, y22]])
    # cuadripolo resultante
    ytn  = yt + yn
    return (ytn)

def printS(S1):
    s11 =   S1[0,0]
    s12 =   S1[0,1]
    s21 =   S1[1,0]
    s22 =   S1[1,1]
    print( 's11= {:4.3f}  {:4.2f}' .format(abs(s11),np.angle(s11)*180/np.pi ))
    print( 's12= {:4.3f}  {:4.2f}' .format(abs(s12),np.angle(s12)*180/np.pi ))
    print( 's21= {:4.3f}  {:4.2f}' .format(abs(s21),np.angle(s21)*180/np.pi ))
    print( 's22= {:4.3f}  {:4.2f}' .format(abs(s22),np.angle(s22)*180/np.pi ))

def printY(Y1):
    y11 =   Y1[0,0]
    y12 =   Y1[0,1]
    y21 =   Y1[1,0]
    y22 =   Y1[1,1]
    print( 'y11= {0.real:4.2E}  {0.imag:4.2E}j' .format(y11) )
    print( 'y12= {0.real:4.2E}  {0.imag:4.2E}j' .format(y12) )
    print( 'y21= {0.real:4.2E}  {0.imag:4.2E}j' .format(y21) )
    print( 'y22= {0.real:4.2E}  {0.imag:4.2E}j' .format(y22) )
    print(' ')

    print( 'r11= {0.real:4.2F}' .format(1/y11.real) )
    print( 'c11= {0.real:4.2E}' .format(y11.imag/wo) )


    print( 'gm = {0.real:4.2F}' .format(abs(y21)) )
    print( 'gr = {0.real:4.4F}' .format(abs(y12)) )

    print( 'r22= {0.real:4.2F}' .format(1/y22.real) )
    print( 'c22= {0.real:4.2E}' .format(y22.imag/wo) )

   

In [57]:
BFP720 = Network('SPICE/BFP720_w_noise_VCE_3.0V_IC_10mA.s2p')

freq = Frequency(10,200,11,'mhz')


In [58]:
fo = 50e6
wo = 2*np.pi*fo

print('Parametros S 50MHz (3V10mA):')

printS(BFP720['50mhz'].s[0])
print(' ')

print('Parametros Y 50MHz (3V10mA):')
Y100 = BFP720['50mhz'].y[0]

printY(Y100)

Parametros S 50MHz (3V10mA):
s11= 0.872  -3.10
s12= 0.002  84.60
s21= 25.105  176.60
s22= 0.995  -1.90
 
Parametros Y 50MHz (3V10mA):
y11= 1.34E-03  2.96E-04j
y12= -1.16E-06  -1.82E-05j
y21= 2.69E-01  -7.78E-03j
y22= 2.78E-05  1.04E-04j
 
r11= 744.33
c11= 9.41E-13
gm = 0.27
gr = 0.0000
r22= 36008.22
c22= 3.31E-13


In [59]:
Linvill(Y100)


###############  LINVILL   #####################
coeficiente de linvill C= 9.26
Es potencialmente inestable


9.2649063671342677

In [60]:
ytn  =  realimenta(Y100,wo,1/1e3 )

printY(ytn)

Linvill(ytn)


y11= 2.34E-03  2.96E-04j
y12= -1.00E-03  -1.82E-05j
y21= 2.68E-01  -7.78E-03j
y22= 1.03E-03  1.04E-04j
 
r11= 426.71
c11= 9.41E-13
gm = 0.27
gr = 0.0010
r22= 972.98
c22= 3.31E-13
###############  LINVILL   #####################
coeficiente de linvill C= 0.98
Es incondicionalmente estable


0.98240244203389149

In [61]:
yg = (1/50 ) - 1.05E-03j
print(1/(1.05E-03*wo) )
print((1.05E-03/wo) )

yl= (1/75 ) - 3.78E-04j
print(1/(3.78E-04*wo) )
print(3.78E-04/wo) 


Stern(yg, yl, ytn)

3.0315227255599115e-06
3.342253804929802e-12
8.420896459888643e-06
1.203211369774729e-12
###############  STERN   #####################
coeficiente de stern K= 40142.13
Estable
yin  = 0.021 4.48E-04
rin  = 47.612 
yout = 0.013 3.78E-04
rout = 76.842 
Gg= 0.51 
G = 220.55 
Gt= 23.43 dB
